# Portfolio Complex Systems Module 10 Empirical Dynamic Modeling

Name: Femian Dooms

Student number: 2086229

Data: Timeseries data featuring power imbalance prices in the Netherlands. Obtained from Tennet transparency (publicaly available).
91 data points measured from 01-01-2024 up untill 01-04-2024. Every datapoint is the mean price at time i where i is measured in days.

Sources:

Contents:

Empirical Dynamic Modeling




In [5]:
import csv
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Create df 
df = pd.read_csv('final_dataset.csv', delimiter=',')
df = df.groupby('Date')[['imbalance_price', 'wind_production', 'solar_production', 'gas_production']].mean().reset_index()



## Intro to EDM
Empirical Dynamic Modeling (EDM) is a data-driven approach used in the field of complex systems analysis to understand and predict the behavior of dynamical systems. It's particularly useful when dealing with nonlinear and complex systems where traditional mathematical modeling techniques may be insufficient.

Here's how it typically works:

Time Series Data Collection: EDM starts with collecting time series data from the system under study. This data could come from various sources such as sensors, experiments, or simulations.

Phase Space Reconstruction: EDM reconstructs the phase space of the system from the time series data. Phase space represents all possible states of a system, and by reconstructing it from the data, EDM aims to capture the underlying dynamics of the system.

Model Construction: Once the phase space is reconstructed, EDM employs various techniques such as Taken's embedding theorem, simplex projection, or delay-coordinate embedding to construct a mathematical model from the data. This model typically takes the form of a set of equations or rules that describe the system's dynamics.

Model Validation and Prediction: The constructed model is then validated using the original data or additional data not used in the model construction. If the model performs well in predicting the behavior of the system, it can be used for prediction and forecasting.

### Steps to code EDM
Data should be sampled at equal intervals
Normalize data to zero mean and unit variance
Should be stationary (detrend if necessary)
Don’t use a linear filter
Be careful of strongly periodic signals

In [32]:
import numpy as np
import pandas as pd
import pyEDM

df['Imbalance_Price_Normalized'] = (df['imbalance_price'] - df['imbalance_price'].mean()) / df['imbalance_price'].std()

# First order change
df['Imbalance_Price_Detrended'] = df['Imbalance_Price_Normalized'].diff()

# Example dataset (replace with your actual data)
time_series_data = {
    "Time": df.index,
    "Driver1": df['solar_production'].values,
    "Target": df['Imbalance_Price_Detrended'].values
}

# Create an EDM object
edm = pyEDM.Simplex()

# Set the columns
target_col = "Target"
driver_cols = "Driver1"
time_col = "Time"

# Load the data
edm.Dataframe(time_series_data, target_col, driver_cols, time_col)

# Embed the data
edm.EmbedDimension(3)  # You can choose the embedding dimension

# Predict using simplex model
prediction = edm.Predict(target_col, 5)  # Predict 5 steps ahead

print("Predicted values:")
print(prediction)

RuntimeError: Validate() Simplex: dataFrame required.

### Distinguishing nonlinear dynamical systems from linear stochastic systems
If nonlinear, low dimensional, and deterministic:
Should be possible to construct a mechanistic model with good forecast skill
High dimensional nonlinear systems can be difficult to distinguish from linear stochastic systems given only time series data
State dependency reflects the degree of nonlinearity

#### S-map
S-Mapping is a technique used in EDM to reconstruct the dynamics of a system. It's based on the concept of finding correlations between the current state of the system and its future states.

Steps to analyse:

Embedding: Similar to other EDM techniques, S-Map starts by embedding the time series data into a high-dimensional space.

Neighborhood Search: For each point in the embedded space, S-Map identifies a neighborhood of nearby points. This neighborhood is typically defined using a distance metric.

Prediction: S-Map uses the data points within each neighborhood to predict the future state of the system. This prediction is typically based on a weighted average of the neighboring points, where the weights are determined by their distances from the current point.

Model Validation: The predicted future states are compared with the actual future states to validate the accuracy of the S-Map model.



In [34]:
from pyEDM import *

# Normalize data to zero mean and unit variance
df =df.drop('Date', axis=1)
normalized_data = (df - df.mean()) / df.std()

# First order
normalized_data['Imbalance_Price_Detrended'] = normalized_data['Imbalance_Price_Normalized'].diff()

# Parameters for EDM
lib = "1 92"  # Embedding library: E1 with embedding dimension 200
pred = "1 10 20 92" 
tau = 1  # Time lag


# Perform Multivariable S-Map
result = SMap(dataFrame=normalized_data, columns=["Imbalance_Price_Detrended", "solar_production"], target="imbalance_price",
              lib=lib, pred=pred, E=200, Tp=1, knn=3, tau=tau, embedded=True)

# Print the resulting dataframe
print(result)


C:\Users\femia\AppData\Local\Temp\ipykernel_43812\661040698.py:15: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  normalized_data = (df - df.mean()) / df.std()
C:\Users\femia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyEDM\EDM.py:248: UserWarning: SMap: CreateIndices():  Disjoint prediction sets not supported.
  warn( msg )


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

## Interpreting the results of the S-map

Interpreting the results of an S-Map analysis involves understanding the relationships between the input variables and the target variable, as well as the predictive power of the model. Here's a general guide on how to interpret the S-Map results:

Pattern Recognition: Look for patterns in the S-Map output. This can include visual inspection of the predicted values compared to the actual values, as well as statistical measures of accuracy.

Prediction Accuracy: Evaluate the accuracy of the S-Map predictions. This can be done by comparing the predicted values to the actual values using metrics such as root mean square error (RMSE), mean absolute error (MAE), or correlation coefficients.

Variable Importance: Assess the importance of each input variable in predicting the target variable. This can be done by examining the weights assigned to each variable in the S-Map model. Variables with higher weights are more influential in predicting the target variable.

Directionality: Determine the directionality of the relationships between variables. In S-Map, positive weights indicate a positive correlation between the input variable and the target variable, while negative weights indicate a negative correlation.

Nonlinear Relationships: Consider the possibility of nonlinear relationships between variables. S-Map is capable of capturing nonlinear dependencies between variables, so it's important to assess whether the relationships identified by the model are linear or nonlinear.

Generalization: Evaluate the generalization of the model to unseen data. This can be done by splitting the data into training and testing sets and assessing the model's performance on the testing set.

Visualization: Use visualizations such as scatter plots, line plots, or heatmaps to explore the relationships between variables and identify any patterns or trends in the data.